In [21]:
%pip install opendatasets
%pip install pandas
import opendatasets as od
import pandas as pd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [22]:
od.download("https://www.kaggle.com/saikrishnadupp/top3000-imdb-indian-movies")
bollywood = pd.read_csv('top3000-imdb-indian-movies/Top3000_imdb_indian_movies.csv')

Skipping, found downloaded files in ".\top3000-imdb-indian-movies" (use force=True to force download)


In [23]:
bollywood

,Title,Date_of_Release,Genre,Crew_dir,Cast_stars,Plot_summary,Age_of_content,Votes,ImDBRating
0,3 Idiots,2009,"Comedy, Drama",Rajkumar Hirani,"Aamir Khan, Madhavan, Mona Singh, Sharman Joshi",Two friends are searching for their long lost ...,12,366038,8.4
1,Taare Zameen Par,2007,"Drama, Family","s:Aamir Khan, Amole Gupte","Darsheel Safary, Aamir Khan, Tisca Chopra, Vip...",An eight-year-old boy is thought to be a lazy ...,14,179838,8.4
2,PK,2014,"Comedy, Drama, Musical",Rajkumar Hirani,"Aamir Khan, Anushka Sharma, Sanjay Dutt, Boman...",An alien on Earth loses the only device he can...,7,171776,8.1
3,Radhe,2021,"Action, Crime, Thriller",Prabhu Deva,"Salman Khan, Disha Patani, Randeep Hooda, Jack...","After taking the dreaded gangster Gani Bhai, A...",0,171349,1.8
4,Dangal,2016,"Action, Biography, Drama",Nitesh Tiwari,"Aamir Khan, Sakshi Tanwar, Fatima Sana Shaikh,...",Former wrestler Mahavir Singh Phogat and his t...,5,170210,8.4
...,...,...,...,...,...,...,...,...,...
2994,Waqt Ki Awaz,1988,"Action, Crime, Drama",K. Bapaiah,"Vikas Anand, Asrani, Mithun Chakraborty, Moush...",Wealthy Lata I. Prasad refuses to obey her wid...,33,105,6.3
2995,Ilzaam,1986,"Drama, Family",Shibu Mitra,"Shashi Kapoor, Shatrughan Sinha, Raj Kiran, An...",Ajay is a street dancer and while people are w...,35,105,5.9
2996,Jheel Ke Us Paar,1973,"Drama, Romance",Bhappi Sonie,"Dharmendra, Mumtaz, Prem Chopra, Mehmood Jr.",Kulwant Rai's horse carriage runs over a child...,48,105,7.2
2997,Ashad Ka Ek Din,1971,"Biography, Drama",Mani Kaul,"Aruna Irani, Arun Khopkar, Rekha Sabnis, Om Sh...",Aashad Ka Ek Din is an adaptation of the play ...,50,105,7.4


DATA PREPROCESSING

In [24]:
bollywood.dropna(subset=['Crew_dir'], inplace=True)
bollywood.drop(columns=['Date_of_Release'], inplace=True)
bollywood.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2994 entries, 0 to 2998
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           2994 non-null   object 
 1   Genre           2994 non-null   object 
 2   Crew_dir        2994 non-null   object 
 3   Cast_stars      2994 non-null   object 
 4   Plot_summary    2994 non-null   object 
 5   Age_of_content  2994 non-null   int64  
 6   Votes           2994 non-null   int64  
 7   ImDBRating      2994 non-null   float64
dtypes: float64(1), int64(2), object(5)
memory usage: 210.5+ KB


In [25]:
bollywood.groupby(['Title']).size().reset_index(name='counts').query('counts > 1') #Probably remakes

,Title,counts
57,Aa Gale Lag Jaa,2
61,Aadmi,2
64,Aag,2
69,Aaina,2
77,Aakrosh,2
...,...,...
2604,Tere Mere Sapne,2
2716,Tumsa Nahin Dekha,2
2720,Udaan,2
2876,Zameer,2


In [26]:
bollywood.describe()

,Age_of_content,Votes,ImDBRating
count,2994.000000,2994.000000,2994.000000
mean,19.763193,4719.244823,5.863427
std,16.149642,13853.006591,1.442464
min,0.000000,105.000000,1.100000
25%,7.000000,257.250000,4.900000
50%,16.000000,772.500000,6.100000
75%,28.000000,3279.500000,7.000000
max,90.000000,366038.000000,9.400000


In [27]:
#JA DZIAŁAM TUTEJ
def find_anomalies(col, dataset, sgm_num = 3):
    col_mean = dataset[col].mean()
    col_std = dataset[col].std()
    col_min = col_mean - sgm_num * col_std
    col_max = col_mean + sgm_num * col_std
    tmp_df = dataset.copy(deep=True)
    tmp_df['col_min'] = col_min
    tmp_df['col_max'] = col_max
    tmp_df = tmp_df.query('' + col + ' > col_max | ' + col + ' < col_min')
    tmp_df = tmp_df.drop(columns=['col_min', 'col_max'])
    return tmp_df

anomalies_votes = find_anomalies('Votes', bollywood)
# anomalies_votes
anomalies_rating = find_anomalies('ImDBRating', bollywood, 2.4)
anomalies_rating.describe()



,Age_of_content,Votes,ImDBRating
count,47.000000,47.000000,47.000000
mean,9.170213,11091.617021,2.219149
std,7.749668,30033.162395,1.117964
min,0.000000,137.000000,1.100000
25%,3.000000,271.000000,1.850000
50%,8.000000,645.000000,2.200000
75%,13.000000,4811.000000,2.350000
max,32.000000,171349.000000,9.400000


In [28]:
# A JA TUTEJ
def split_actors(df_o):
    df = df_o.copy(deep=True)
    actors =  df['Cast_stars']
    df['Cast_stars'] = actors.map(lambda x: x.split(', '))
    df = df.explode('Cast_stars')
    return df
bollywood_actors = split_actors(bollywood)
bollywood_actors.describe()

,Age_of_content,Votes,ImDBRating,col_min,col_max
count,11954.000000,11954.000000,11954.000000,11954.000000,11954.000000
mean,19.786933,4726.834867,5.861277,2.401512,9.325341
std,16.149573,13862.823809,1.442561,0.000000,0.000000
min,0.000000,105.000000,1.100000,2.401512,9.325341
25%,7.000000,259.000000,4.900000,2.401512,9.325341
50%,16.000000,776.000000,6.100000,2.401512,9.325341
75%,28.000000,3296.000000,7.000000,2.401512,9.325341
max,90.000000,366038.000000,9.400000,2.401512,9.325341
